<a href="https://colab.research.google.com/github/vggls/msc_thesis_medical_xai/blob/main/experiments/ham10k_resnet50/HAAS_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Imports**

In [ ]:
pip install grad-cam

In [ ]:
import torch
from torchvision  import datasets, transforms
from pytorch_grad_cam import GradCAM, HiResCAM

In [ ]:
# custom written code
from ham10k import map_images_to_paths, create_datasets, summaries
from haas import Haas, HA_image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **2. Download from Kaggle**

In [ ]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download kmader/skin-cancer-mnist-ham10000

100% 5.18G/5.20G [00:33<00:00, 198MB/s]
100% 5.20G/5.20G [00:33<00:00, 167MB/s]


In [ ]:
! unzip skin-cancer-mnist-ham10000.zip

# **3. Test Dataset**

In [ ]:
# to be applied to training data
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=20),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])
# to be applied to validation and test data
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])

In [ ]:
dataframe = map_images_to_paths()

In [ ]:
train_dataset, validation_dataset, test_dataset= create_datasets(dataframe = dataframe,
                                                                 train_transforms = train_transforms,
                                                                 test_transforms = test_transforms)

No lesion overlap among training, validation and test datasets!
All images succesfully distributed in the training, validation and test datasets!


In [ ]:
# do not need them in this notebook
import shutil
shutil.rmtree('./training_dataset')
shutil.rmtree('./validation_dataset')

# **4. ResNet50**

In [ ]:
drive_path = './drive/MyDrive/Colab_Notebooks/dataset_models/HAM10000/Models/'
resnet50 = torch.load(drive_path + 'ham10k_resnet50.pt')
resnet50 = resnet50.cuda()
resnet50 = resnet50.eval()

In [ ]:
resnet50_gradcam_instance = GradCAM(model=resnet50, target_layers=[resnet50.layer4[2].conv3], use_cuda=True)
resnet50_hirescam_instance = HiResCAM(model=resnet50, target_layers=[resnet50.layer4[2].conv3], use_cuda=True)

print('-->GradCAM')
gradcam_score, _, _, _ = Haas(test_dataset, resnet50, resnet50_gradcam_instance)
print('HAAS score:', gradcam_score)
print('---------------------------')
print('-->HiResCAM')
hirescam_score, _, _, _ = Haas(test_dataset, resnet50, resnet50_hirescam_instance)
print('HAAS score:', hirescam_score)

-->GradCAM
No. of test images:  1992
Total time: 117.77222919464111 secs
Accuracy over original images:  0.705
Accuracy over HA images:  0.586
HAAS score: 0.831
---------------------------
-->HiResCAM
No. of test images:  1992
Total time: 113.22008562088013 secs
Accuracy over original images:  0.705
Accuracy over HA images:  0.585
HAAS score: 0.83
